In [124]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

#Snorkel
import re
import string
import pandas as pd
from snorkel.labeling import LabelingFunction
from snorkel.preprocess import preprocessor
from snorkel.labeling import PandasLFApplier
from snorkel.labeling.model import LabelModel
from snorkel.labeling import LFAnalysis
from snorkel.labeling import filter_unlabeled_dataframe
from snorkel.labeling import labeling_function

pd.set_option('display.max_colwidth', None)

### Loading and Preprocessing Textual Data From Twitter and News Headlines

#### Loading DataFrames

In [29]:
# Twitter
df_twitter = pd.read_pickle('../crawlers/twitter/twitter_df.pkl')
df_twitter.rename(columns={'text':'title'}, inplace=True)
df_twitter = df_twitter[['title']]
df_twitter['source'] = 'Twitter'
df_twitter.index = df_twitter.index.rename('date')
df_twitter.sample(5)

,title,source
date,,
2022-02-25 14:52:02,"Credit Suisse reduz preço-alvo da Azul (AZUL4), mas mantém recomendação\nhttps://t.co/kPS1EwFflP",Twitter
2022-04-14 13:12:55,"""Choque macrofinanceiro"" à vista em caso de ruptura comercial Rússia-Alemanha, segundo S&amp;P - https://t.co/AXDSdvveUe",Twitter
2021-10-01 09:16:22,BIOMM - Acordo exclusivo para fornecimento da vacina Convidecia no Brasil.\n#biomm https://t.co/c0H0bwOy9t,Twitter
2021-10-24 19:05:29,Estamos ao vivo: https://t.co/JmjEnkURY6 https://t.co/GWcl9LEqvo https://t.co/Q0EuRbrfOZ,Twitter
2022-04-26 10:10:20,Vale (VALE3) quer descarbonizar operação de siderurgia; entenda ‘novo passo ESG’ - https://t.co/3UBBCKbLov,Twitter


In [30]:
# News Headlines
df_news = pd.read_pickle('../crawlers/suno/news_df.pkl')
df_news = df_news[['title']]
df_news['source'] = 'Suno'
df_news.sample(5)

,title,source
date,,
2021-05-15 11:29:00,Ibovespa: Yduqs (YDUQ3) e Petrobras (PETR4) lideram altas na semana,Suno
2019-10-24 19:32:00,"Vale registra lucro de R$ 6,461 bilhões no 3T19",Suno
2021-10-05 19:23:00,Petróleo fecha em alta e WTI alcança recorde histórico pelo segundo dia consecutivo,Suno
2021-06-04 10:48:00,Vale (VALE3) interrompe atividades no Complexo Mariana,Suno
2021-05-31 19:49:00,Petrobras (PETR4) conclui venda da Eólica Mangue Seco 2,Suno


In [82]:
# Append all dataframes

df_full = pd.DataFrame()

frames = [df_twitter, df_news]

for df in frames:
    df_full = df_full.append(df)

df_full.sample(5)

,title,source
date,,
2022-04-04 07:48:37,CPF negativado: saiba como liquidar as dívidas e limpar seu cadastro\n#investimentos #mercadofinanceiro #leiamoneytimes\nhttps://t.co/NKP2zfmkjU,Twitter
2022-02-22 12:12:56,"Bitcoin (BTC) segue mercado de ações e recua, repercutindo aumento das tensões entre Rússia e Ocidente\nhttps://t.co/3lO8cGsuAZ",Twitter
2022-03-30 08:53:02,"Para especialistas, mudanças na Petrobras põem gestão em xeque\nhttps://t.co/140ozRgVfc",Twitter
2022-03-07 09:33:52,Abertura de mercado: o que esperar para bolsa e câmbio no Brasil nesta 2ª-feira - https://t.co/fz2x6e5Rnh,Twitter
2022-03-08 11:42:15,"EUA devem suspender importações de petróleo da Rússia, dizem agências https://t.co/M6l9tQIaaz",Twitter


In [83]:
df_full.source.value_counts()

Twitter    19985
Suno        2650
Name: source, dtype: int64

#### Preprocessing Data

In [45]:
def remove_emojis(sentence):

    "Remoção de Emojis nas mensagens de texto."

    # Padrões dos Emojis
    emoji_pattern = re.compile("["
                u"\U0001F600-\U0001F64F"  # emoticons
                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                u"\U00002702-\U000027B0"
                u"\U000024C2-\U0001F251"
                u"\U0001f926-\U0001f937"
                u'\U00010000-\U0010ffff'
                u"\u200d"
                u"\u2640-\u2642"
                u"\u2600-\u2B55"
                u"\u23cf"
                u"\u23e9"
                u"\u231a"
                u"\u3030"
                u"\ufe0f"
    "]+", flags=re.UNICODE)

    return emoji_pattern.sub(r'', sentence)

def remove_valores(sentence):
    new_sentece = ''
    
    for token in sentence.split():
        if token.isdigit():
            token = '<NUM>'
        new_sentece += ' {}'.format(token)
        
    return new_sentece

In [159]:
# 1. Aplicar preprocessamento nos títulos e textos completos

def apply_preprocessing(df_input: pd.DataFrame) -> pd.DataFrame:

    df = df_input.copy()

    # Substituir símbolos importantes
    df['title'] = df['title'].map(lambda s: s.replace('-feira', ''))
    df['title'] = df['title'].map(lambda s: s.replace('-alvo', ' alvo'))
    df['title'] = df['title'].map(lambda s: s.replace('+', ''))
    df['title'] = df['title'].map(lambda s: s.replace('-', ''))
    df['title'] = df['title'].map(lambda s: s.replace('%', ' por cento'))
    df['title'] = df['title'].map(lambda s: s.replace('R$', ''))
    df['title'] = df['title'].map(lambda s: s.replace('U$', ''))
    df['title'] = df['title'].map(lambda s: s.replace('US$', ''))
    df['title'] = df['title'].map(lambda s: s.replace('S&P 500', 'spx'))
    df['title'] = df['title'].map(lambda s: s.replace('/', '@'))

    # Remove Links, Hashtags e Menções
    df['title'] = df['title'].str.replace('http.*\s',"")
    df['title'] = df['title'].str.replace('(\#\w+.*?)',"")
    df['title'] = df['title'].str.replace('(\@\w+.*?)',"")

    # Transformar em String e Letras Minúsculas nas Mensagens
    df['title'] = df['title'].map(lambda s: str(s).lower())

    # Remover Pontuações
    df['title'] = df['title'].map(lambda s: s.translate(str.maketrans('', '', string.punctuation)))

    # Remover Emojis     
    df['title'] = df['title'].map(lambda s: remove_emojis(s))

    # Quebras de Linha desnecessárias
    df['title'] = df['title'].map(lambda s: s.replace('\n', ' '))

    # Remover aspas duplas
    df['title'] = df['title'].map(lambda s: s.replace('\"', ''))
    df['title'] = df['title'].map(lambda s: s.replace('“', ''))
    df['title'] = df['title'].map(lambda s: s.replace('”', ''))

    # Remover valores
    df['title'] = df['title'].map(lambda s: remove_valores(s))

    # Espaços desnecessários
    df['title'] = df['title'].map(lambda s: s.strip())

    return df

In [160]:
# d = {'title': ['Truist corta preço-alvo da Netflix @NTFLIX #follow - https://t.co/9nECLjknvA']}
# df_test = pd.DataFrame(d)

df_full_preprocessed = apply_preprocessing(df_full)
df_full_preprocessed.sample(5)

,title,source
date,,
2022-03-22 14:57:08,índia vai taxar ganhos com criptomoedas em <NUM> por cento httpsco,Twitter
2022-03-02 12:43:06,wall st sobe após powell dizer que aumentos de juros seguem nos trilhos httpsco,Twitter
2021-07-06 20:27:06,foram realizados <NUM> leilões na b3 no primeiro semestre de <NUM> leilões são processos licitatórios em que o poder público concede ativos à iniciativa privada mas você sabe por que a bolsa do brasil está relacionada a esses processos entenda no httpsco,Twitter
2019-05-17 20:56:00,se é para caixa dar lucro privatiza logo diz paulo guedes,Suno
2022-04-14 13:58:40,rússia status livre de armas nucleares do mar báltico está em perigo httpsco,Twitter


In [113]:
df_full.loc['2022-04-11 20:53:50']

,title,source
date,,
2022-04-11 20:53:50,Truist corta preço-alvo da Netflix - https://t.co/9nECLjknvA,Twitter


#### Create Train/Test Splits

### Creating Labelling Functions (LFs)

In [5]:
POSITIVE = 1
NEGATIVE = 0
ABSTAIN = -1

categories = [POSITIVE, NEGATIVE, ABSTAIN]

#### Positive Sentiment LFs

- Adjectives
- Verbs

In [6]:
@labeling_function()
def lf_news_good_adjs(x):
    with open('./dicts/final/pos_adj.txt') as file:
        adjectives = [line.rstrip() for line in file]
    
    # Identificação dos adjetivos na frase
    for word in x.title.lower().split():
        if word in adjectives:
            return POSITIVE
    return ABSTAIN

@labeling_function()
def lf_news_good_verbs(x):
    with open('./dicts/pos_verbs.txt') as file:
        verbs = [line.rstrip() for line in file]
    
    for word in x.title.lower().split():
        if word in verbs:
            return POSITIVE
        
    return ABSTAIN

@labeling_function()
def lf_regex_dividendos(x):
    dividend_pattern = r".*pag.*dividendo.*|.*anunc.*dividendo.*|.*distrib.*dividendo.*"
    return POSITIVE if re.search(dividend_pattern, x.title.lower(), flags=re.I) else ABSTAIN

@labeling_function()
def lf_regex_resultado_positivo(x):
    raise_pattern = r"fech.*alta.*|.*abr.*alta.*|.*fech.*pos.*|.*abr.*pos.*|.*estre.*alta.*|.*prev.*alta.*" 
    return POSITIVE if re.search(raise_pattern, x.title.lower(), flags=re.I) else ABSTAIN

#### Negative Sentiment LFs

In [9]:
@labeling_function()
def lf_news_bad_adjs(x):
    # Lista Inicial de Adjetivos 
    with open('./dicts/final/neg_adj.txt') as file:
        adjectives = [line.rstrip() for line in file]

    # Identificação dos adjetivos na frase
    for word in x.title.lower().split():
        if word in adjectives:
            return NEGATIVE

    return ABSTAIN

@labeling_function()
def lf_news_bad_verbs(x):
    with open('./dicts/neg_verbs.txt') as file:
        verbs = [line.rstrip() for line in file]
    
    for word in x.title.lower().split():
        if word in verbs:
            return NEGATIVE
        
    return ABSTAIN

@labeling_function()
def lf_regex_resultado_negativo(x):
    fall_pattern = r"fech.*queda.*|.*abr.*queda.*|.*fech.*neg.*|.*abr.*neg.*|.*prev.*baixa.*|.*prev.*queda.*|.*em.*queda.*" 
    
    if re.search(fall_pattern, x.title.lower(), flags=re.I):
        return NEGATIVE
    else:
        return ABSTAIN
    
    # return NEGATIVE if re.search(fall_pattern, x.title.lower(), flags=re.I) else ABSTAIN

In [8]:
df = pd.read_pickle('../crawlers/twitter/twitter_df.pkl')

In [15]:
df.rename(columns={'text': 'title'}, inplace=True)

In [16]:
df.sample(1)

,tweet_id,search_dt,title,user_id,screen_name,rt_count,favorite_count
created_at,,,,,,,
2022-04-05 07:59:35,1511297492507312132,2022-05-03 14:16:39,"Economia da zona do euro tem impulso em março com reabertura, mas preços avançam, mostra PMI\n#investimentos #mercadofinanceiro #leiamoneytimes\nhttps://t.co/BYmEZEldgp",51150679,leiamoneytimes,0,1


In [17]:
# combine all the labeling functions 
lfs = [
       lf_news_good_adjs, 
       lf_news_good_verbs,
       lf_regex_dividendos,
       lf_regex_resultado_positivo,
       lf_news_bad_adjs,
       lf_news_bad_verbs,
       lf_regex_resultado_negativo]

# apply the lfs on the dataframe
applier = PandasLFApplier(lfs=lfs)
L_snorkel = applier.apply(df=df, progress_bar=False)

# apply the label model
label_model = LabelModel(cardinality=len(categories), device='cpu', verbose=False)

# fit on the data
label_model.fit(L_snorkel)

# predict and create the labels
df['label'] = label_model.predict(L=L_snorkel, tie_break_policy='abstain').astype(str)

# convert to classes
dict_map = {'-1': 'NEUTRAL', '1': 'POSITIVE', '0': 'NEGATIVE'}
df['label_class'] = df['label'].map(dict_map)

100%|██████████| 100/100 [00:00<00:00, 564.20epoch/s]


In [22]:
df[['title', 'label_class', 'screen_name']].sample(10)

,title,label_class,screen_name
created_at,,,
2022-03-11 08:51:54,"Lucro da Santos Brasil (STBP3) soma R$ 113,8 mi no 4T21; alta de 695% - https://t.co/iWbFmIPFi2",NEUTRAL,sunonoticias
2022-04-23 14:31:57,"Com gasolina cara, mercado de bairro ganha a preferência https://t.co/trJnAZlQJG",NEUTRAL,infomoney
2018-09-13 18:50:39,"Sonia Favaretto, from B3, spoke at the PRI In Person Conference. The PRI is the largest responsible investment initiative in the world aimed at promoting the #ESG agenda. Learn more about B3's participation in the event and the topics discussed. @PRI_News @CFAinstitute https://t.co/UgFL8rRsIw",NEUTRAL,B3_Oficial
2022-03-30 08:08:01,"Ataque hacker rouba R$ 3 bilhões em criptomoedas da Ronin, do Axie Infinity - https://t.co/W1cxScRk8s",NEUTRAL,sunonoticias
2022-04-24 16:22:31,Emmanuel Macron ganha segundo mandato nas eleições presidenciais da França\nhttps://t.co/lL2nKMcJ0k,NEGATIVE,valorinveste
2022-04-20 11:05:36,Dólar oscila ante real de olho em fraqueza da moeda no exterior; política monetária domina foco\n#investimentos #mercadofinanceiro #leiamoneytimes\nhttps://t.co/BVGY9lUibJ,POSITIVE,leiamoneytimes
2022-04-18 13:23:09,Mirae Asset troca quatro ações em carteira recomendada para aumentar rentabilidade - https://t.co/NCOdc8kkAD,NEUTRAL,InvestingBrasil
2022-03-15 17:59:01,"Clientes relatam sumiço de dinheiro na NuInvest, que fala em problema resolvido - https://t.co/8s1x61CdnU \n\n$NU #NUBR33 #Nubank",NEUTRAL,InvestingBrasil
2022-03-17 17:47:35,Dólar e juros futuros fecham em queda com fluxo externo e Copom \nhttps://t.co/pSPiQIM4AX,NEGATIVE,valorinveste


In [23]:
#Filtering out unlabeled data points
df = df.loc[df.label_class.isin(['POSITIVE', 'NEGATIVE']), :]

print ('Quantidade Total de Mensagens Rotuladas: ', df.shape[0])

# find the label counts 
df['label_class'].value_counts() / df.shape[0] * 100

Quantidade Total de Mensagens Rotuladas:  6049


POSITIVE    67.680608
NEGATIVE    32.319392
Name: label_class, dtype: float64